# Download & import library

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints using Mediapipe

In [2]:
# set up the mediapipe holistic model and drawing, holistic model is 
# the integration of face, hand, body etc model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connec

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Extract Keypoint

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Set up Folder for data collection

In [7]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(['normal','hello', 'peace', 'salute'])
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Keypoint for Training & Testing

In [52]:
cap = cv2.VideoCapture(0)    #access the webcam, 0 mean the first camera device found in computer
# set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # loop through action
    for action in actions:
        #loop through video length wif respect to videos
        for sequence in range(no_sequences):
            # loop through video length wif respect to sequence length / frame
            for frame_num in range(sequence_length):
                
                return_value, frame = cap.read()

                # make prediction
                image, results = mediapipe_detection(frame, holistic)

                # draw landmarks
                draw_styled_landmarks(image, results)
                
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):   # if 'q' key on the keyboard is pressed, break the loop
                    break
    cap.release()
    cv2.destroyAllWindows()

# Data Preprocessing

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
actions = np.array(['normal','hello', 'peace', 'salute'])
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'normal': 0, 'hello': 1, 'peace': 2, 'salute': 3}

In [10]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Build LSTM model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [60]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [61]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [62]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=150, callbacks=[tb_callback])

Epoch 1/150


4/4 [==============================] - 4s 31ms/step - loss: 1.5631 - categorical_accuracy: 0.3246
Epoch 2/150
4/4 [==============================] - 0s 28ms/step - loss: 3.7494 - categorical_accuracy: 0.2456
Epoch 3/150
4/4 [==============================] - 0s 28ms/step - loss: 1.3430 - categorical_accuracy: 0.3860
Epoch 4/150
4/4 [==============================] - 0s 33ms/step - loss: 1.5420 - categorical_accuracy: 0.3421
Epoch 5/150
4/4 [==============================] - 0s 29ms/step - loss: 1.7921 - categorical_accuracy: 0.2105
Epoch 6/150
4/4 [==============================] - 0s 33ms/step - loss: 1.6425 - categorical_accuracy: 0.2895
Epoch 7/150
4/4 [==============================] - 0s 30ms/step - loss: 1.3134 - categorical_accuracy: 0.2807
Epoch 8/150
4/4 [==============================] - 0s 29ms/step - loss: 1.2222 - categorical_accuracy: 0.4649
Epoch 9/150
4/4 [==============================] - 0s 30ms/step - loss: 1.0447 - categorical_accuracy: 0.5439
Epoch 10

4/4 [==============================] - 0s 29ms/step - loss: 0.0048 - categorical_accuracy: 1.0000
Epoch 144/150
4/4 [==============================] - 0s 29ms/step - loss: 0.0018 - categorical_accuracy: 1.0000
Epoch 145/150
4/4 [==============================] - 0s 32ms/step - loss: 0.0027 - categorical_accuracy: 1.0000
Epoch 146/150
4/4 [==============================] - 0s 38ms/step - loss: 0.0022 - categorical_accuracy: 1.0000
Epoch 147/150
4/4 [==============================] - 0s 36ms/step - loss: 0.0024 - categorical_accuracy: 1.0000
Epoch 148/150
4/4 [==============================] - 0s 29ms/step - loss: 0.0014 - categorical_accuracy: 1.0000
Epoch 149/150
4/4 [==============================] - 0s 29ms/step - loss: 0.0017 - categorical_accuracy: 1.0000
Epoch 150/150
4/4 [==============================] - 0s 32ms/step - loss: 0.0011 - categorical_accuracy: 1.0000


In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 596708 (2.28 MB)
Trainable params: 596708 

# Save Weights (model)

In [64]:
model.save('model2.keras')

# Load model

In [14]:
from keras.models import load_model
model = load_model('model2.keras')

# Confusion Matrix

In [15]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [16]:
yhat = model.predict(X_train)
yhat = np.argmax(yhat, axis=1).tolist()
ytrue = np.argmax(y_train, axis=1).tolist()

4/4 [==============================] - 1s 11ms/step


In [17]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[87,  0],
        [ 0, 27]],

       [[85,  0],
        [ 0, 29]],

       [[84,  0],
        [ 0, 30]],

       [[86,  0],
        [ 0, 28]]], dtype=int64)

In [18]:
accuracy_score(ytrue, yhat)

1.0

# Real time Test

In [19]:
res = model.predict(X_test)

1/1 [==============================] - 0s 21ms/step


In [20]:
res

array([[5.7995212e-08, 1.1870533e-07, 6.3419582e-05, 9.9993634e-01],
       [9.9993122e-01, 1.8527435e-06, 2.7366536e-07, 6.6630222e-05],
       [8.9348866e-08, 1.9472155e-07, 9.9573197e-05, 9.9990010e-01],
       [9.9991679e-01, 3.3284114e-06, 2.7104008e-06, 7.7144541e-05],
       [9.9992156e-01, 1.6486025e-06, 4.3425164e-08, 7.6781660e-05],
       [1.4569959e-07, 9.9987912e-01, 1.2071280e-04, 5.7822572e-08]],
      dtype=float32)

In [21]:
actions = np.array(['normal','hello', 'peace', 'salute'])
colors = [(245,117,16), (117,245,16), (16,117,245), (245,117,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [22]:
res=model.predict(np.expand_dims(X_test[0], axis=0))[0]
res

1/1 [==============================] - 0s 20ms/step


array([5.7995212e-08, 1.1870510e-07, 6.3419342e-05, 9.9993634e-01],
      dtype=float32)

In [23]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]


        # Viz probabilities
        image = prob_viz(res, actions, image, colors)
            
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step


...

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 17ms/step


In [76]:
cap.release()
cv2.destroyAllWindows()